In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import euclidean


In [2]:
df = pd.read_csv("preprocessed_data.csv")
df.head()

,Unnamed: 0,id,popularity,release_date,revenue,runtime,title,vote_average,vote_count,tags_vector
0,0,19995,150.437577,2009-12-10,2787965087,162.0,Avatar,7.2,11800,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,285,139.082615,2007-05-19,961000000,169.0,Pirates of the Caribbean: At World's End,6.9,4500,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2,206647,107.376788,2015-10-26,880674609,148.0,Spectre,6.3,4466,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,3,49026,112.312950,2012-07-16,1084939099,165.0,The Dark Knight Rises,7.6,9106,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,4,49529,43.926995,2012-03-07,284139100,132.0,John Carter,6.1,2124,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [3]:
df.drop('Unnamed: 0' , axis = 1 , inplace = True)

In [4]:
df.head()

,id,popularity,release_date,revenue,runtime,title,vote_average,vote_count,tags_vector
0,19995,150.437577,2009-12-10,2787965087,162.0,Avatar,7.2,11800,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,285,139.082615,2007-05-19,961000000,169.0,Pirates of the Caribbean: At World's End,6.9,4500,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,206647,107.376788,2015-10-26,880674609,148.0,Spectre,6.3,4466,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,49026,112.312950,2012-07-16,1084939099,165.0,The Dark Knight Rises,7.6,9106,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,49529,43.926995,2012-03-07,284139100,132.0,John Carter,6.1,2124,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Calculates similarities using:
Content (tags) similarity: 20%
Popularity: 20%
Vote average: 20%
Other factors (revenue, runtime, recency, vote count): 40%

In [10]:
import pandas as pd
import numpy as np

def create_movie_recommender(df):
    movies = df.copy()
    scaler = MinMaxScaler()
    
    # Normalize features
    features = ['vote_average', 'vote_count', 'popularity', 'revenue', 'runtime']
    movies[features] = scaler.fit_transform(movies[features])
    
    # Convert tags_vector strings to arrays if needed
    if isinstance(movies['tags_vector'].iloc[0], str):
        movies['tags_vector'] = movies['tags_vector'].apply(eval)
    
    def get_movie_similarity(row1, row2):
        features_distance = euclidean(
            row1[features].values,
            row2[features].values
        )
        
        tag_similarity = 1 - euclidean(
            np.array(row1['tags_vector']),
            np.array(row2['tags_vector'])
        )
        
        return features_distance + (1 - tag_similarity)
    
    def get_recommendations(movie_id, n=5):
        target_movie = movies[movies['id'] == movie_id].iloc[0]
        movies['similarity'] = movies.apply(lambda x: get_movie_similarity(x, target_movie), axis=1)
        return movies.nsmallest(n+1, 'similarity')[1:][['title', 'vote_average', 'popularity', 'revenue', 'runtime']]
    
    return get_recommendations

In [12]:
recommender = create_movie_recommender(df)
similar_movies = recommender(19995)  # For Avatar
print(similar_movies)

                      title  vote_average  popularity   revenue   runtime
16             The Avengers          0.74    0.164974  0.545042  0.423077
25                  Titanic          0.75    0.114239  0.661785  0.573964
31               Iron Man 3          0.68    0.088720  0.435960  0.384615
7   Avengers: Age of Ultron          0.73    0.153360  0.504097  0.417160
28           Jurassic World          0.65    0.478206  0.542879  0.366864
